In [1]:
import exploratory_analysis

import json
import uuid
import re
import pickle
import pandas as pd
import numpy as np

import csv

import copy

import plotly.express as px

sys.path.append('src/')
import preprocessing
import get_ent_info
import exploratory_analysis
import get_reference

%load_ext autoreload
%autoreload 2


 (title, author, publication date and revision date)

In [2]:
with open('get_data/sep_entry_notext_list.json', 'r') as f:
    ent_list = json.load(f)

update_ent_dict = {}
for item in ent_list:
    update_item  = copy.deepcopy (item)
    url = item['url']
    title = item['title']
    authorship = item['authorship']
    pubinfo = item['pubinfo']
    bibliography = item['bibliography']['categories']
    related_entries = item['related_entries']

    # %%%%%%%%%%%% "ent_type"
    update_item['ent_type'] = ''
    update_item['type_parent'] = ''

    # %%%%%%%%%%%% update the authorship
    # they seem to be all divided by "\n"
    # e.g., "Peter King\n\nAndrew Arlig\n<aarlig@brooklyn.cuny.edu>"
    update_item['authorship']['author_list'] = [i for i in authorship['author_text'].split("\n") if i and not i.startswith("<")]
    # %%%%%%%%%%%% update the pub year
    # all (1769) entries have "First published" 
    # 1462 entries have "substantive revision"
    # the variable only have either 1 or 2 items in the list
    update_item['pubinfo'] = {
        'first_published_date': " ".join(pubinfo[0].split(" ")[-3:]),
        'first_published_year': pubinfo[0].split(" ")[-1],
        'substantive_revision_date': "",
        'substantive_revision_year': ""
    }
    if len(pubinfo) >= 2:
        update_item['pubinfo']['substantive_revision_date'] = " ".join(pubinfo[1].split(" ")[-3:])
        update_item['pubinfo']['substantive_revision_year'] = pubinfo[1].split(" ")[-1]

    # %%%%%%%%%%%% clean

    update_item['related_entries']['entry_link'] = [list(i.keys()) for i in update_item['related_entries']['entry_link']]
    update_item['related_entries']['entry_link'] = [re.findall(r'../(.*?)/', s, re.DOTALL)[0] for sublist in update_item['related_entries']['entry_link'] for s in sublist]
    
    update_item['authorship'] = update_item['authorship']['author_list']
    update_item['bibliography'] = bibliography
    update_item['academic_tools'] = []

    # remove other information
    update_item['other_internet_resources'] = []

    # ******************* Update the dictionary *****************
    update_ent_dict[url] = update_item

# update_ent_dict["enlightenment"]


# save
with open('database/entry_meta_dict.json', 'w') as f:
    json.dump(update_ent_dict, f, indent=4)

## Add toc

In [3]:
# open get_data/toc/ent_toc_dict.json
with open('get_data/toc/ent_toc_dict.json', 'r') as f:
    ent_toc_dict = json.load(f)

# open entry_meta_dict.json
with open('database/entry_meta_dict.json', 'r') as f:
    ent_meta_dict = json.load(f)

update_ent_dict = {}
# add toc to the ent_meta_dict
for url in list(ent_meta_dict.keys()):
    update_item  = copy.deepcopy(ent_meta_dict[url])
    
    for ent_url in list(ent_toc_dict.keys()):
        if ent_url == url:
            update_item['toc'] = ent_toc_dict[ent_url]["toc_nest_dict"]

    update_ent_dict[url] = update_item

# save
with open('database/entry_meta_dict.json', 'w') as f:
    json.dump(update_ent_dict, f, indent=4)


FileNotFoundError: [Errno 2] No such file or directory: 'get_data/toc/ent_toc_dict.json'

## Distinguish between thinker and thoughts?


thinkers (recognized philosophers), thoughts of a thinker, or philosophy theories 

In [4]:
# open entry_meta_dict.json
with open('database/entry_meta_dict.json', 'r') as f:
    entry_meta_dict = json.load(f)
# open stanford_encyclopedia\data\sep_network.json
with open('meta_data/sep_network.json', 'r', encoding='utf-8') as f:
    sep_network = json.load(f)

# find thinkers' url in stanford_encyclopedia\data\sep_network.json
thinker_url_list = []
ent_list = sep_network["articles"]["nodes"]
for ent_dict in ent_list:
    if ent_dict["entry_type"] == "thinker":
        thinker_url_list.append("".join(ent_dict["id"].split("/")[-2:]))

# print(thinker_url_list[0])

# add "type" = "thinker" to entry_meta_dict.json
# else is "thought"

count_thinker = 0
count_thought = 0
for ent_url in entry_meta_dict:
    if ent_url in thinker_url_list:
        entry_meta_dict[ent_url]['ent_type'] = 'thinker'
        count_thinker += 1
    else:
        entry_meta_dict[ent_url]['ent_type'] = 'thought'
        count_thought += 1


# print("count_thinker: ", count_thinker)
# print("count_thought: ", count_thought)

# save entry_meta_dict.json
with open('database/entry_meta_dict.json', 'w') as f:
    json.dump(entry_meta_dict, f, indent=4)


### Find entries of a philosopher's work

**Pattern of the url:**

- url of the thinker entry: kant, then we find entry: Kant’s Philosophy of Mathematics (kant-mathematics)
- missing cases: Monroe Beardsley does not have an entry, while "Beardsley’s Aesthetics " exists

In [ ]:
"Russell\u2019s Logical Atomism".lower()

'russell’s logical atomism'

In [5]:
# open entry_meta_dict.json
with open('database/entry_meta_dict.json', 'r') as f:
    ent_meta_dict = json.load(f)

# find entries whoee titles ca=ontain thinkers name
# thinkers names are titles of entries whose type is thinker
# e.g., "Hume’s Aesthetics"  or "Hume on Free Will"
# not: "James of Viterbo" (james-viterbo) ≠ "William James" (james)
thinker_url_list = []
for ent_url in ent_meta_dict:
    if ent_meta_dict[ent_url]['ent_type'] == 'thinker':
        thinker_name = ent_meta_dict[ent_url]['title']
        thinker_url_list.append(ent_url)


for ent_url in ent_meta_dict:
    url = ent_meta_dict[ent_url]['url']
    title = ent_meta_dict[ent_url]['title']
    
    if ent_meta_dict[ent_url]['ent_type'] == "thought":
        if ("’s " in title or "s’ " in title or " on " in title):
            find_thinker = False
            for thinker_url in thinker_url_list:
                thinker_name = ent_meta_dict[thinker_url]['title']
                last_name = thinker_name.split(" ")[-1]
                # print(last_name)
                if thinker_url+"-" in url:
                    find_thinker = True
                    # print(thinker_url+"==========")
                    # print(ent_meta_dict[ent_url]['title'])
                    # print(ent_meta_dict[ent_url]['url'])
                    ent_meta_dict[ent_url]['ent_type'] = 'thought of thinker'
                    ent_meta_dict[ent_url]['type_parent'] = ent_meta_dict[thinker_url]['title']
                elif title.lower().split("\u2019s")[0] in thinker_name.lower() or last_name.lower()+" on " in title.lower(): 
                    # Locke on Real Essence
                    find_thinker = True
                    # print(thinker_url+"==========")
                    # print(ent_meta_dict[ent_url]['title'])
                    # print(ent_meta_dict[ent_url]['url'])
                    print(thinker_name, title)
                    ent_meta_dict[ent_url]['ent_type'] = 'thought of thinker'
                    ent_meta_dict[ent_url]['type_parent'] = ent_meta_dict[thinker_url]['title']
                    
            if find_thinker == False:
                ent_meta_dict[ent_url]['ent_type'] = 'thought of thinker'
                # ent_meta_dict[ent_url]['type_parent'] = ''
                print("NOT FOUND=================="+url+"==================="+title+"")
        else:
            ent_meta_dict[ent_url]['ent_type'] = 'topic'
            print("=================="+url+"==================="+title+"")

update_ent_meta_dict = copy.deepcopy(ent_meta_dict)


with open('database/entry_meta_dict.json', 'w') as f:
    json.dump(update_ent_meta_dict, f, indent=4)

==================simulations-science===================Computer Simulations in Science
==================testimony-episprob===================Epistemological Problems of Testimony
==================loyalty===================Loyalty
==================it-privacy===================Privacy and Information Technology
==================computational-complexity===================Computational Complexity Theory
==================functionalism===================Functionalism
==================social-choice===================Social Choice Theory
==================moral-luck===================Moral Luck
==================legitimacy===================Political Legitimacy
==================theater===================Philosophy of Theater
==================human-genome===================The Human Genome Project
==================naturalism-india===================Naturalism in Classical Indian Philosophy
==================moral-theory===================Moral Theory
==================symmetry-breakin

In [6]:

with open('database/entry_meta_dict.json', 'r') as f:
    ent_meta_dict = json.load(f)
# print all "thought of thinker" entries that do not have a "type_parent"
for ent_url in ent_meta_dict:
    if ent_meta_dict[ent_url]['ent_type'] == "thought of thinker":
        if ent_meta_dict[ent_url]['type_parent'] == '' and "Femini" not in ent_meta_dict[ent_url]['title']:
            print(ent_meta_dict[ent_url]['title'])
            print()
            

Robert Nozick’s Political Philosophy

Curry’s Paradox

Pufendorf’s Moral and Political Philosophy

Adam Smith’s Moral and Political Philosophy

Croce’s Aesthetics

Boltzmann’s Work in Statistical Physics

Zermelo’s Axiomatization of Set Theory

Bayes’ Theorem

Molyneux’s Problem

Children’s Rights

Dedekind’s Contributions to the Foundations of Mathematics

Influence of Arabic and Islamic Philosophy on the Latin West

Simpson’s Paradox

Influence of Arabic and Islamic Philosophy on Judaic Thought

Hilbert’s Program

Prisoner’s Dilemma

Skolem’s Paradox

Beardsley’s Aesthetics

Bohr’s Correspondence Principle

Arrow’s Theorem

Bell’s Theorem

Fitch’s Paradox of Knowability

Joseph Butler’s Moral Philosophy

Teleological Arguments for God’s Existence



In [7]:
# check whether author names are missing
with open('database/entry_meta_dict.json', 'r') as f:
    ent_meta_dict = json.load(f)

miss_author_cat_dict = {}
# print all the entries whose type is "thought of thinker"
# and those type is "thinker"
# print out the bibliography of these entries
for ent_url in ent_meta_dict:
    if ent_meta_dict[ent_url]['ent_type'] == 'thought of thinker' or ent_meta_dict[ent_url]['ent_type'] == 'thinker':
        last_name = ""
        thinker_name = ""
        # note that the title may not always be the thinker name, e.g., "Godfrey of Fontaines" refered as "Works by Godfrey"
        if ent_meta_dict[ent_url]['ent_type'] == 'thought of thinker':
            thinker_name = ent_meta_dict[ent_url]['type_parent']
            if thinker_name:
                last_name = [i for i in thinker_name.split(" ") if i.strip() != ""][-1]
        else:
            thinker_name = ent_meta_dict[ent_url]['title']
            last_name = [i for i in thinker_name.split(" ") if i.strip() != ""][-1]

        # if not last_name:
        #     print("last_name is empty", ent_meta_dict[ent_url]['title'])

        if ent_meta_dict[ent_url]['bibliography']:
            for cat in ent_meta_dict[ent_url]['bibliography']:
                cat_lower = " "+cat.lower()+" "
                # ERROR : "al-Farabi" "Works of Al-Fārābī" "al-farabi-logic"
                # or, "Baron de Montesquieu, Charles-Louis de Secondat" => "Montesquieu"
                # in this case, we check url, because it is very likely to be the most refered name

                # CASE 3: find cat that contains thinker's last name
                # "Wundt: Selected Publications"

                # sometimes is first name: "Modern Editions of James\u2019 Works"

                # CASE 2: "Primary Sources" "Primary Literature" "Primary sorces for Laibniz"
                if "primary" in cat_lower.split(" "):
                    print("primary", "===========",cat)
                    miss_author_cat_dict[ent_url+cat] = thinker_name
                elif "abbreviations" in cat_lower or "collected works" in cat_lower:
                    print("abbreviations", "===========",cat)
                    miss_author_cat_dict[ent_url+cat] = thinker_name
                elif " translation" in cat_lower or " edition" in cat_lower or " in German" in cat or " in Greek" in cat or "in English" in cat:
                    print("translations", "===========",cat)
                    miss_author_cat_dict[ent_url+cat] = thinker_name
                elif "work by " in cat.lower() or "works by " in cat.lower():
                    print("work by", "===========",cat)
                    miss_author_cat_dict[ent_url+cat] = thinker_name
                elif last_name and last_name.lower() in cat_lower and not "secondary" in cat_lower:
                    # "Books about Scholem"
                    # "Related Works on Hume and His Context" "Other works by Du Bois referred to in the text"
                    if " by " in cat_lower.split(last_name.lower())[0] or last_name.lower()+"\u2019s" in cat_lower or last_name.lower()+"'s" in cat_lower:
                        print("NAME:", last_name, "===========", cat)
                        miss_author_cat_dict[ent_url+cat] = thinker_name
                
                # "Editions and Translations of His Commentaries" "Abbreviations"
                # "Works of \u015a\u0101ntideva"

                # CASE 1: if "Work by" or "Works by" or "\u2019s Works" use the title of the entry as author name
                # elif " work " in cat_lower or " works " in cat_lower:
                #     print("work", "===========",cat)
                
                # "Nishida Kitar\u014d" "Nishida\u2019s works"
                elif "\u2019s work" in cat_lower or "'s work" in cat_lower:
                    # print("check this" , cat, ent_url, "|", thinker_name)
                    # print(cat_lower.split("\u2019s work")[0])
                    if cat_lower.split("\u2019s work")[0].strip() in thinker_name.lower() or cat_lower.split("'s work")[0].strip() in thinker_name.lower():
                        print("not cited as ln", "====", thinker_name, "===========",cat)
                        miss_author_cat_dict[ent_url+cat] = thinker_name
                
                url_name = ent_url.replace("-"," ")
                if url_name in cat_lower and not "secondary" in cat_lower:
                    # "Books about Scholem"
                    # "Related Works on Hume and His Context" "Other works by Du Bois referred to in the text"
                    if " by " in cat_lower.split(url_name.lower())[0] or url_name.lower()+"\u2019s" in cat_lower or url_name.lower()+"'s" in cat_lower:
                        print("NAME:", url_name, "===========", cat)
                        miss_author_cat_dict[ent_url+cat] = thinker_name
                
                
                # print(ent_meta_dict[ent_url]['title'], ent_meta_dict[ent_url]['url'])
                # print(ent_meta_dict[ent_url]['bibliography'])
# save miss_author_cat_list
with open('get_data/to_check/miss_author_cat_dict.json', 'w') as f:
    json.dump(miss_author_cat_dict, f, indent=4)


translations =========== Reinhold’s Works in German and English in Chronological Order
primary =========== Primary Sources
primary =========== Primary Literature
primary =========== Primary Sources
primary =========== Primary Literature
work by =========== Works by Poulain
primary =========== Primary Literature
work by =========== Works by Telesio
NAME: telesio =========== Works by Telesio
primary =========== Primary Sources
primary =========== Wittgenstein's Work: Primary Sources
primary =========== Primary Literature
primary =========== Primary Literature: Works by Marcel
NAME: marcel =========== Primary Literature: Works by Marcel
NAME: Stein =========== A. Stein’s Work (German and English)
NAME: stein =========== A. Stein’s Work (German and English)
primary =========== Primary Literature
translations =========== Individual Works and English translations
primary =========== Primary Sources
NAME: Kant =========== Recent Commentaries on Kant’s Ethical Writings
primary =========== Prim

In [ ]:
# # Add entry type information to 'database/sep_clean/ent_ref_sep_dict.json'
# with open('database/entry_meta_dict.json', 'r') as f:
#     ent_meta_dict = json.load(f)
# with open('database/sep_clean/ent_ref_sep_dict.json', 'r') as f:
#     ent_ref_sep_dict = json.load(f)
# ent_ref_sep_dict['abelard']
# # for ent_url in ent_ref_sep_dict:
# #     ent_ref_sep_dict[ent_url]['ent_type'] = ent_meta_dict[ent_url]['ent_type']
# #     ent_ref_sep_dict[ent_url]['type_parent'] = ent_meta_dict[ent_url]['type_parent']
# #     # add table of contents
# #     ent_ref_sep_dict[ent_url]['toc'] = ent_meta_dict[ent_url]['toc']

{'Primary texts in Latin': {'ori_ref_count': 29,
  'ref_count': 29,
  'ref_dict': {'1932eb84-1a1b-4b40-817b-5319ef7e3c58': {'ref': '. Edited by J. M. A.\nRubingh-Bosscher in Peter Abelard: Carmen ad Astralabium, a\nCritical Edition. Groningen: phil. diss. Rijksuniversiteit\n1987.',
    'title': 'Carmen ad Astralabium',
    'parent_title': '',
    'title_in_title': 'Carmen ad Astralabium',
    'year': '1987',
    'author_list': [],
    'author': '',
    'update_author_list': [],
    'update_ref': 'Carmen ad Astralabium (1987) []'},
   'd62bffa2-9030-4e29-99db-24651c12f898': {'ref': 'Dialogus inter Philosophum,\nIudaeum, et Christianum. Edited by Giovanni Orlandi, with\nintroduction, translation, and notes by John Marenbon, in Peter\nAbelard: Collationes, Oxford University Press 2001.',
    'title': 'Collationes',
    'parent_title': '',
    'title_in_title': 'Collationes',
    'year': '2001',
    'author_list': [],
    'author': '',
    'update_author_list': [],
    'update_ref': 'Colla

In [8]:
# prepare to transfroming into csv
# don't keep nested json


# update the toc in ent_toc_dict
with open('get_data/toc/ent_toc_dict.json', 'r') as f:
    ent_toc_dict = json.load(f)

with open('database/entry_meta_dict.json', 'r') as f:
    ent_meta_dict = json.load(f)

# to update the toc in ent_meta_dict
for ent_url in ent_meta_dict:
    if ent_url in ent_toc_dict:
        ent_meta_dict[ent_url]['toc'] = ent_toc_dict[ent_url]['toc_list']


# from "pubinfo",
# add "first_published_date", "first_published_year", "substantive_revision_date", "substantive_revision_year" as individual keys
for ent_url in ent_meta_dict:
    if "pubinfo" in ent_meta_dict[ent_url]:
        pubinfo = ent_meta_dict[ent_url]['pubinfo']
        # print(pubinfo)
        if "first_published_date" in pubinfo:
            ent_meta_dict[ent_url]['first_published_date'] = pubinfo['first_published_date']
        if "first_published_year" in pubinfo:
            ent_meta_dict[ent_url]['first_published_year'] = pubinfo['first_published_year']
        if "substantive_revision_date" in pubinfo:
            ent_meta_dict[ent_url]['substantive_revision_date'] = pubinfo['substantive_revision_date']
        if "substantive_revision_year" in pubinfo:
            ent_meta_dict[ent_url]['substantive_revision_year'] = pubinfo['substantive_revision_year']
        # del ent_meta_dict[ent_url]['pubinfo']




# ent_meta_dict['abelard']
with open('database/entry_meta_dict.json', 'w') as f:
    json.dump(ent_meta_dict, f, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: 'get_data/toc/ent_toc_dict.json'

In [ ]:
ent_meta_dict['abelard'].keys()

dict_keys(['url', 'title', 'authorship', 'pubinfo', 'preamble', 'toc', 'bibliography', 'related_entries', 'academic_tools', 'other_internet_resources', 'ent_type', 'type_parent', 'first_published_date', 'first_published_year', 'substantive_revision_date', 'substantive_revision_year'])

to csv

In [ ]:
# convert entry_meta_dict.json to csv
# open entry_meta_dict.json
with open('database/entry_meta_dict.json', 'r') as f:
    entry_meta_dict = json.load(f)

# convert entry_meta_dict.json to csv
# first column is the url, second is title, third is authorship, fourth is pubinfo, fifth is bibliography, sixth is related_entries, seventh is toc, eighth is ent_type
# with open('database/entry_meta_dict.csv', 'w', newline='', encoding='UTF-8') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(["url", "title", "authorship", "pubinfo", "bibliography", "related_entries", "toc", "ent_type"])
#     for url in entry_meta_dict:
#         writer.writerow([url, entry_meta_dict[url]['title'], entry_meta_dict[url]['authorship'], entry_meta_dict[url]['pubinfo'], entry_meta_dict[url]['bibliography'], entry_meta_dict[url]['related_entries']['entry_link'], entry_meta_dict[url]['toc'], entry_meta_dict[url]['ent_type']])


# dict to df, where list is written in one cell
df = pd.DataFrame.from_dict(entry_meta_dict, orient='index')
# take toc as a list
df_toc = list(df['toc'])

del df['toc']

df['authorship'] = df['authorship'].apply(lambda x: "; ".join(x))

# ['related_entries'] shuold be ['related_entries']['entry_link']
df['related_entries'] = df['related_entries'].apply(lambda x: x['entry_link'])
# pubinfo should be separated into four columns
df['first_published_date'] = df['pubinfo'].apply(lambda x: x['first_published_date'])
df['first_published_year'] = df['pubinfo'].apply(lambda x: x['first_published_year'])
df['substantive_revision_date'] = df['pubinfo'].apply(lambda x: x['substantive_revision_date'])
df['substantive_revision_year'] = df['pubinfo'].apply(lambda x: x['substantive_revision_year'])

df = df.reset_index()
# df = df.rename(columns={'index': 'url'})

df['ent_type'] = df['ent_type'].apply(lambda x: x if x else np.nan)
# if empty, fill with NaN
df['type_parent'] = df['type_parent'].apply(lambda x: x if x else np.nan)


df['related_entries'] = df['related_entries'].apply(lambda x: "; ".join(x))
# df['toc'] = df['toc'].apply(lambda x: "; ".join(x))
df['bibliography'] = df['bibliography'].apply(lambda x: "; ".join(x))
df['academic_tools'] = df['academic_tools'].apply(lambda x: ", ".join(x))
df['other_internet_resources'] = df['other_internet_resources'].apply(lambda x: ", ".join(x))


# here, add toc_list as a new column
df['toc'] = df_toc
# .apply(lambda x: "; ".join(x))


# delete the first column of this df
del df['index']
del df['pubinfo']

df['toc'] 

# save df as csv
df.to_csv('database/entry_meta.csv', index=False)

## We can try to identify some specific categories of entries 

based on content

For example, feminist philosophy and women thinkers

In [12]:
# with open('meta_data/sep_network.json', 'r', encoding='utf-8') as f:
#     sep_network = json.load(f)

with open('database/entry_meta_dict.json', 'r') as f:
    ent_meta_dict = json.load(f)

women_url_list = []

potential_url_list, tocheck_url_list = [], []
matches = [" she ", ". She ", " her ", ". Her"]
caution = [" he ", ". He ", " his ", ". His"]

cheked_list = ["Śrīharṣa", "Judah Abrabanel", "Johann Friedrich Herbart", "Francis Herbert Bradley", "David Hartley", "Count Paul Yorck von Wartenburg"]

for ent_dict in ent_meta_dict.values():
    if ent_dict["ent_type"] == "thinker":
        ent_url = ent_dict["url"]
        preamble_text = ent_dict["preamble"]
        if any(x in preamble_text for x in matches):
            # if any(x in preamble_text for x in caution):
            #     tocheck_url_list.append(ent_url)
            #     print("*********", ent_dict["title"])
            #     print(preamble_text,"\n")
            # else:
            if ent_dict["title"] not in cheked_list:
                # potential_url_list.append(ent_url)
                women_url_list.append(ent_url)
                # print("=========", ent_dict["title"])
                # print(preamble_text,"\n")

women_url_list.append('murdoch')
# print length
print(len(women_url_list))


ent_meta_cat_dict = ent_meta_dict.copy()
# update the dict for entries whose ent_type is thinker
for k, v in ent_meta_cat_dict.items():
    v["ent_category"] = []
    if v["ent_type"] == "thinker":
        # v["gender_sex"] = ""
        if k in women_url_list:
            # v["ent_category"].append("women_thinker")
            v["ent_category"] = "women_thinker"


print(ent_meta_cat_dict['anna-julia-cooper']['ent_category'])
print(ent_meta_cat_dict['plato']['ent_category'])

# print entry titles have "feminis"
# “Feminist” or “Feminism”
feminist_url_list, feminism_url_list = [],[]

for k, v in ent_meta_dict.items():
    if "feminism" in v["title"].lower():
        # print(v["title"])
        feminism_url_list.append(k)
        # v["ent_category"].append("feminism")
        v["ent_category"] = "feminism"

    if "feminist" in v["title"].lower():
        # print(v["title"])
        feminist_url_list.append(k)
        # v["ent_category"].append("feminist")
        v["ent_category"] = "feminist"

print(len(feminist_url_list), len(feminism_url_list))

          
# save the updated dict
with open('database/entry_meta_cat_dict.json', 'w') as f:
    json.dump(ent_meta_cat_dict, f, indent=4)


27
women_thinker
[]
30 9


chinese philosophy

In [17]:
from bs4 import BeautifulSoup

with open('database/entry_meta_cat_dict.json', 'r') as f:
    ent_meta_dict = json.load(f)

chinese_phil_url_list = []

html_content = """
<li> Chinese Philosophy
   <ul>
   <li> <a href="entries/buddhism-chan/"><strong>Chan Buddhism</strong></a> (Peter Hershock) </li>
   <li> <a href="entries/chinese-phil-medicine/"><strong>Chinese medicine</strong></a> (Lisa Raphals) </li>
   <li> Compared to Western Philosophy — see <a href="entries/comparphil-chiwes/">comparative philosophy: Chinese and Western</a> </li>
   <li> Confucius — see <a href="entries/confucius/">Confucius</a> </li>
   <li> Daoism (Taoism) — see <a href="entries/daoism/">Daoism</a> </li>
   <li> <a href="entries/emotions-chinese/"><strong>emotions in early Chinese Philosophy</strong></a> (Bongrae Seok) </li>
   <li> <a href="entries/chinese-epistemology/"><strong>epistemology</strong></a> (Jana Rošker) </li>
   <li> <a href="entries/ethics-chinese/"><strong>ethics</strong></a> (David Wong) </li>
   <li> <a href="entries/han-dynasty/"><strong>Han Dynasty</strong></a> (Alexus McLeod) </li>
   <li> <a href="entries/buddhism-huayan/"><strong>Huayan Buddhism</strong></a> (Bryan Van Norden and Nicholaos Jones) </li>
   <li> Laozi — see <a href="entries/laozi/">Laozi</a> </li>
   <li> <a href="entries/chinese-legalism/"><strong>legalism in</strong></a> (Yuri Pines) </li>
   <li> <a href="entries/chinese-logic-language/"><strong>logic and language in Early Chinese Philosophy</strong></a> (Marshall D. Willman) </li>
   <li> Mencius — see <a href="entries/mencius/">Mencius</a> </li>
   <li> <a href="entries/chinese-metaphysics/"><strong>metaphysics</strong></a> (Franklin Perkins) </li>
   <li> <a href="entries/chinese-mind/"><strong>mind (heart-mind)</strong></a> (David Wong) </li>
   <li> <a href="entries/mohism/"><strong>Mohism</strong></a> (Chris Fraser) </li>
   <li> <a href="entries/mohist-canons/"><strong>Mohist Canons</strong></a> (Chris Fraser) </li>
   <li> Neo-Daoism — see <a href="entries/neo-daoism/">Neo-Daoism</a> </li>
   <li> <a href="entries/chinese-change/"><strong>philosophy of change</strong></a> (Tze-Ki Hon) </li>
   <li> <a href="entries/qing-philosophy/"><strong>Qing philosophy</strong></a> (On-cho Ng) </li>
   <li> <a href="entries/chinese-phil-science/"><strong>science</strong></a> (Lisa Raphals) </li>
   <li> <a href="entries/chinese-social-political/"><strong>social and political thought</strong></a> (Stephen C. Angle) </li>
   <li> <a href="entries/song-ming-confucianism/"><strong>Song-Ming Confucianism</strong></a> (Justin Tiwald) </li>
   <li> <a href="entries/buddhism-tiantai/"><strong>Tiantai Buddhism</strong></a> (Brook Ziporyn) </li>
   <li> <a href="entries/chinese-translate-interpret/"><strong>translating and interpreting</strong></a> (Henry Rosemont Jr. and Chad Hansen) </li>
   <li> Wang Yangming — see <a href="entries/wang-yangming/">Wang Yangming</a> </li>
   <li> Xunzi — see <a href="entries/xunzi/">Xunzi</a> </li>
   <li> Zhuang Zi — see <a href="entries/zhuangzi/">Zhuangzi</a> </li>
   <li> Zhu Xi — see <a href="entries/zhu-xi/">Zhu Xi</a> </li>
   </ul></li>
"""

# Parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Extract all <a> elements with href attributes containing "entry/"
entries = soup.find_all('a', href=lambda href: href and "entries/" in href)

# Print the extracted IDs from hrefs
for entry in entries:
    href = entry['href']
    id = href.replace('entries/', '').rstrip('/')
    # print(id)
    chinese_phil_url_list.append(id)

for k, v in ent_meta_dict.items():
    if v["url"] in chinese_phil_url_list:
        v["ent_category"] = "chinese_phil"
        print(v["title"])

# save the updated dict
with open('database/entry_meta_cat_dict.json', 'w') as f:
    json.dump(ent_meta_dict, f, indent=4)

Daoism
Legalism in Chinese Philosophy
Metaphysics in Chinese Philosophy
Song-Ming Confucianism
Chinese Ethics
The Emotions in Early Chinese Philosophy
Mohism
Tiantai Buddhism
Chan Buddhism
Mohist Canons
Huayan Buddhism
Logic and Language in Early Chinese Philosophy
Philosophy in Han Dynasty China
Chinese Philosophy of Change (Yijing)
Xunzi
Science and Chinese Philosophy
Zhu Xi
Epistemology in Chinese Philosophy
Qing Philosophy
Laozi
Confucius
Neo-Daoism
Chinese Philosophy and Chinese Medicine
Translating and Interpreting Chinese Philosophy
Mencius
Wang Yangming
Zhuangzi
Mind (Heart-Mind) in Chinese Philosophy
Social and Political Thought in Chinese Philosophy
Comparative Philosophy: Chinese and Western


In [ ]:
# print entry titles have "feminis"
# “Feminist” or “Feminism”
feminist_url_list, feminism_url_list = [],[]

for k, v in ent_meta_dict.items():
    if "feminist" in v["title"].lower():
        print("------", v["title"])
        feminist_url_list.append(k)
    if "feminism" in v["title"].lower():
        print("======", v["title"])
        feminism_url_list.append(k)
print(len(feminist_url_list), len(feminism_url_list))

------ Feminist Metaphysics
------ Feminist Philosophy of Language
------ Feminist Perspectives on Autonomy
====== The History of Feminism: Marie-Jean-Antoine-Nicolas de Caritat, Marquis de Condorcet
------ Feminist Perspectives on the Self
------ Feminist Ethics
====== Pragmatist Feminism
====== Liberal Feminism
------ Feminist Epistemology and Philosophy of Science
====== Psychoanalytic Feminism
------ Feminist Philosophy of Law
------ Feminist Perspectives on Rape
====== Analytic Feminism
------ Feminist Perspectives on Sex and Gender
------ Feminist Aesthetics
------ Feminist Perspectives on Class and Work
------ Feminist Perspectives on Power
------ Feminist Perspectives on Globalization
------ Feminist Perspectives on Argumentation
------ Feminist Environmental Philosophy
------ Feminist Moral Psychology
------ Feminist Perspectives on Trans Issues
====== Continental Feminism
------ Feminist Philosophy
------ Feminist Political Philosophy
------ Feminist Perspectives on the Body


In [ ]:
# feminism-related entries should not be considered as "thought of thinker"

# open ent_meta_dict
with open('database/entry_meta_dict.json', 'r') as f:
    ent_meta_dict = json.load(f)

for k, v in ent_meta_dict.items():
    if k in feminist_url_list or k in feminism_url_list:
        v["ent_type"] = "topic"
        # print(v["ent_type"])
        # print(v["title"])
    if v['title'] == "Iris Murdoch":
        v["gender_sex"] = "female"
        v["ent_type"] = "thinker"



with open('database/entry_meta_dict.json', 'w') as f:
    json.dump(ent_meta_dict, f, indent=4)

In [ ]:
def add_trace_line_chart(target_list, target_list_name, target):
    

    # get the year list
    year_list = []
    for k, v in ent_meta_dict.items():
        if k in target_list:
            if target == "published":
                year_list.append(int(list(v["pubinfo"].values())[1]))
            elif target == "revised":
                try:
                    year_list.append(int(list(v["pubinfo"].values())[3]))
                except:
                    pass

    year_count_list = []
    for year in range(1990, 2023):
        year_count_list.append(year_list.count(year))


    fig.add_trace(go.Scatter(x=list(range(1990, 2023)), y=year_count_list,
                        mode='lines', #+markers',
                        name= target_list_name + target))

In [ ]:
%run get_ent_info.py

fem_related_list = []
for i in feminist_url_list:
    fem_related_list.append(get_ent_title(i))


with open("database/fem_related_list.json", "w") as f:
    json.dump(fem_related_list, f, indent=4)

KeyboardInterrupt: 

NameError: name 'get_ent_title' is not defined

In [ ]:

fem_conbined_list = women_url_list + feminist_url_list + feminism_url_list

# open database/thinker_linked_women_list.json
with open('database/thinker_linked_women_list.json', 'r', encoding='utf-8') as f:
    thinker_linked_women_list = json.load(f)

# for entries in these lists, print the pubinfo
# for k, v in ent_meta_dict.items():
#     if k in fem_conbined_list:
#         print("------", v["title"])
#         print(list(v["pubinfo"].values())[1], list(v["pubinfo"].values())[3])


# https://plotly.com/python/line-charts/

import plotly.graph_objects as go
# use plotlly to draw line chart of the number of entries over time, based on v["pubinfo"].values())[1]
# draw the line chart
fig = go.Figure()

add_trace_line_chart(women_url_list, "Feminism-related | ", "published")
add_trace_line_chart(women_url_list, "Feminism-related | ", "revised")
add_trace_line_chart(thinker_linked_women_list, "Linked to w/f | ", "published")
add_trace_line_chart(thinker_linked_women_list, "Linked to w/f | ", "revised")
add_trace_line_chart(fem_conbined_list, "Women philosophers | ", "published")
add_trace_line_chart(fem_conbined_list, "Women philosophers | ", "revised")
# add_trace_line_chart(list(ent_meta_dict.keys()), "all | ", "published")
# add_trace_line_chart(list(ent_meta_dict.keys()), "all | ", "revised")


fig.update_layout(title='Number of entries over time',
                    xaxis_title='Year',
                    yaxis_title='Number of entries')
# show each year
fig.update_xaxes(tickmode = 'linear')
# set the range of y axis
fig.update_yaxes(range=[-1, 30])
# make figures higher
fig.update_layout(height=700)

# make smooth lines more visible 

fig.update_traces(line=dict(width=1.5))
# transparent background
fig.update_layout(plot_bgcolor='rgba(10,10,10,0)')
# make traces more transparent
fig.update_traces(opacity=0.8)

# size
fig.update_layout(
    autosize=False,
    width=1000,
    height=400)

# make y-axis more dense, only show 5, 10, 50, 100, 200, 300
# fig.update_yaxes(tickmode = 'array', tickvals = [0, 5, 10, 50, 100, 200, 300])
fig.show()


In [ ]:

%run get_ent_info.py
fem_related_list = feminist_url_list + feminism_url_list
fem_related_list = [get_ent_title(i) for i in fem_related_list]

# save as database/fem_related_list.json
with open("database/fem_related_list.json", "w") as f:
    json.dump(fem_related_list, f, indent=4)

women_thinker_list = [get_ent_title(i) for i in women_url_list]

with open("database/women_thinker_list.json", "w") as f:
    json.dump(women_thinker_list, f, indent=4)